## **Tratando a base**


### Dicionário de dados:
#### Disponível em <https://www.gov.br/prf/pt-br/acesso-a-informacao/dados-abertos/dicionario-acidentes>
* **id**: Variável com valores numéricos, representando o identificador do acidente;
* **data_inversa**: Data da ocorrência no formato dd/mm/aaaa;
* **dia_semana**: Dia da semana da ocorrência. Ex.: Segunda, Terça, etc;
* **horario**: Horário da ocorrência no formato hh:mm:ss;
* **uf**: Unidade da Federação. Ex.: MG, PE, DF, etc;
* **br**: Variável com valores numéricos representando o identificador da BR do acidente;
* **km**: Identificação do quilômetro onde ocorreu o acidente, com
valor mínimo de 0,1 km e com a casa decimal separada
por ponto;
* **municipio**: Nome do município de ocorrência do acidente;
* **causa_acidente**: Identificação da causa presumível do acidente. Ex.: Falta de atenção, Velocidade incompatível, etc;
* **tipo_acidente**: Identificação do tipo de acidente. Ex.: Colisão frontal, Saída de pista,etc;
* **classificacao_acidente**: Classificação quanto à gravidade do acidente: Sem Vítimas, Com Vítimas Feridas, Com Vítimas Fatais e Ignorado; 
* **fase_dia**: Fase do dia no momento do acidente. Ex. Amanhecer, Pleno dia, etc;
* **sentido_via**: Sentido da via considerando o ponto de colisão: Crescente e decrescente;
* **condicao_meteorologica**: Condição meteorológica no momento do acidente: Céu claro, chuva,vento,etc;
* **tipo_pista**: Tipo da pista considerando a quantidade de faixas:Dupla,simples ou múltipla;
* **tracado_via**: Descrição do traçado da via: reta, curva ou cruzamento;
* **uso_solo**: Descrição sobre as características do local do acidente: Urbano=Sim ou Rural=Não;
* **ano**: Ano da ocorrência;
* **pessoas**: Total de pessoas envolvidas na ocorrência;
* **mortos**: Total de pessoas mortas envolvidas na ocorrência;
* **feridos_leves**: Total de pessoas com ferimentos leves envolvidas na ocorrência;
* **feridos_graves**: Total de pessoas com ferimentos graves envolvidas na ocorrência;
* **ilesos**: Total de pessoas ilesas envolvidas na ocorrência;
* **ignorados**: Total de pessoas envolvidas na ocorrência e que não se soube o estado físico;
* **feridos**: Total de pessoas feridas envolvidas na ocorrência (é a soma dos feridos leves com os graves);
* **veiculos**: Total de veículos envolvidos na ocorrência;
* **latitude**: Latitude do local do acidente em formato geodésico
decimal;
* **longitude**: Longitude do local do acidente em formato geodésico
decimal;
* **regional**: - 
* **delegacia**: -
* **uop**: -

In [1]:
import pandas as pd
import numpy as np
import math as mt
from unicodedata import normalize

In [ ]:
# Datasets concatenation

datasets = [f'/content/drive/MyDrive/PosPuc/TCC/datasets/acidentes_ocorrencia/datatran{i}.csv' for i in range(2007, 2022)]
df_geral = pd.DataFrame()

for data in datasets:
    df_temp = pd.read_csv(data, sep=';', encoding='utf-8')
    df_geral = pd.concat([df_geral, df_temp], ignore_index=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Copy dataframe
df = df_geral.copy()

In [ ]:
# Dataset shape
df.shape

In [ ]:
# Dataset check columns types
df.info()

In [ ]:
# Using missingno package to check dataset health
import missingno as msno
msno.matrix(df)

In [ ]:
# Dataset head / tail
df.head()

In [ ]:
df.tail()

In [ ]:
# Check ID duplicity
id_duplicity = df.groupby('id', as_index=False).agg({'id': ['first', 'count']})
ids_duplicity = id_duplicity[id_duplicity[('id', 'count')] > 1][('id', 'first')].values.tolist()
len(ids_duplicity)

In [ ]:
df[df.id == ids_duplicity[0]]

In [ ]:
# Columns NaN
df[(df.br == '(null)') | (df.br.isna())]

### **Conclusions**
* Must be necessary:
    * Convert feature **id** from float to str;
    * Fill features **br**, **km**, **regional**, **delegacia** and **uop**;
    * Convert feature **br** from float to str;
    * Truncate feature **km**;
    * Create a interval of 50km from feature **km**, eg: if km = 67 the interval must be 50 - 100;
    * Adjust feature **data_inversa** to format aaaa-mm-dd;
    * Adjust feature **dia_semana** to only first name and capitalize, eg: SABADO, SEGUNDA, etc;
    * Remove accentuation from features **dia_semana**, **causa_acidente**, **tipo_acidente**, **classificacao_acidente**, **fase_dia**, **condicao_metereologica**, **tipo_pista**, **tracado_via** e **uso_solo**;
    * Capitalize features **dia_semana**, **causa_acidente**, **tipo_acidente**, **classificacao_acidente**, **fase_dia**, **sentido_via**, **condicao_metereologica**, **tipo_pista**, **tracado_via** e **uso_solo**;
    * Fill feature **ano** with respective year where is null and convert to int;
    * Check duplicity into **id** column, some duplications came from same id but different years;
    * Features **latitude**, **longitude** are written with ' , ' change to ' . ' and convert to float;
    * Check features **latitude**, **longitude** and fill nan values;
    * Add **mes** field;




In [ ]:
# Convert features id
df['id'] = df.id.astype(int).astype(str)

In [ ]:
# Fillna features br, km, regional, delegacia and uop
values = {'br': '0', 'km': '-1', 'regional': 'UNKNOWN', 'delegacia': 'UNKNOWN', 'uop': 'UNKNOWN'}
df.fillna(values, inplace=True)

In [ ]:
# Convert features br and km
df.loc[df.br == '(null)', 'br'] = '0'
df['br'] = df.br.astype(int).astype(str)

df.loc[df.km == '(null)', 'km'] = '-1'
df['km'] = df.apply(lambda x: x.km.replace(',', '.') if isinstance(x.km, str) else x.km, axis=1)
df['km'] = df.km.astype(float).astype(int)

In [ ]:
# Create a km_interval
def gen_interval(x, num_interval):
    int_min = x - x % num_interval
    int_max = int_min + num_interval
    return f'[{int_min} - {int_max}]'

df['km_intervalo'] = df.apply(lambda x: gen_interval(x.km, 50), axis=1)

In [ ]:
# Adjust feature data_inversa to format aaaa-mm-dd
df['data_inversa'] = pd.to_datetime(df.data_inversa, dayfirst=True)

In [ ]:
# Adjust feature dia_semana to only first name and capitalize, eg: SABADO, SEGUNDA, etc
df['dia_semana'] = df.apply(lambda x: x.dia_semana.split('-')[0].upper(), axis=1)

In [ ]:
# Remove nan values
fill_value = '(null)'

df.fillna({'dia_semana': fill_value,
           'causa_acidente': fill_value, 
           'tipo_acidente':fill_value, 
           'classificacao_acidente': fill_value, 
           'fase_dia': fill_value, 
           'condicao_metereologica': fill_value, 
           'tipo_pista': fill_value, 
           'tracado_via': fill_value, 
           'uso_solo': fill_value}, inplace=True)

# Remove accentuation and capitalize strings
def remove_accents(x):
    return normalize('NFKD', x).encode('ASCII', 'ignore').decode('ASCII').upper()

df = (
    df
    .assign(dia_semana = df.apply(lambda x: remove_accents(x.dia_semana), axis=1))
    .assign(causa_acidente = df.apply(lambda x: remove_accents(x.causa_acidente), axis=1))
    .assign(tipo_acidente = df.apply(lambda x: remove_accents(x.tipo_acidente), axis=1))
    .assign(classificacao_acidente = df.apply(lambda x: remove_accents(x.classificacao_acidente), axis=1))
    .assign(fase_dia = df.apply(lambda x: remove_accents(x.fase_dia), axis=1))
    .assign(condicao_metereologica = df.apply(lambda x: remove_accents(x.condicao_metereologica), axis=1))
    .assign(tipo_pista = df.apply(lambda x: remove_accents(x.tipo_pista), axis=1))
    .assign(tracado_via = df.apply(lambda x: remove_accents(x.tracado_via), axis=1))
    .assign(uso_solo = df.apply(lambda x: remove_accents(x.uso_solo), axis=1))
)

In [ ]:
# Fill feature ano
df['ano'] = df.apply(lambda x: x.data_inversa.year, axis=1)

In [ ]:
# Check ID duplicity
id_duplicity = df.groupby(['id', 'ano'], as_index=False).agg({'id': ['first', 'count'], 'ano': 'first'})
ids_duplicity = id_duplicity[id_duplicity[('id', 'count')] > 1][('id', 'first')].values.tolist()
print(f' Was found: {len(ids_duplicity)} duplicate ids')

# Remove duplicity
df.drop_duplicates(subset=['id', 'ano'], keep='first', inplace=True)

In [ ]:
# Change longitude and latitude fields type
df['longitude'] = df.apply(lambda x: x.longitude if isinstance(x.longitude, float) else x.longitude.replace(',', '.'), axis=1)
df['latitude'] = df.apply(lambda x: x.latitude if isinstance(x.latitude, float) else x.latitude.replace(',', '.'), axis=1)

df['longitude'] = df['longitude'].astype(float)
df['latitude'] = df['latitude'].astype(float)

In [ ]:
# Fill latitude and longitude
df_temp = (
    df
    .groupby(['br', 'km_intervalo'], as_index=False)
    .agg({'longitude': 'mean', 'latitude': 'mean'})
).rename(columns={'longitude': 'temp_long', 'latitude': 'temp_lati'}, inplace=True)

# Joining tables
df = df.merge(df_temp, on=['br', 'km_intervalo'], how='left')

df = (
    df
    .assing(longitude = df.apply(lambda x: x.temp_long if (np.isnan(x.longitude) and not np.isnan(x.temp_long)) else 0, axis=1))
    .assing(latitude = df.apply(lambda x: x.temp_lati if (np.isnan(x.latitude) and not np.isnan(x.temp_lati)) else 0, axis=1))
)

# Remove unnecessary columns
df.drop(columns=['temp_long', 'temp_lati'], inplace=True)

In [ ]:
# Add feature mes
df['mes'] = df.apply(lambda x: x.data_inversa.month, axis=1)

In [ ]:
# Final dataset health
import missingno as msno
msno.matrix(df)

In [ ]:
# Save dataframe preprocessed
df.to_csv('/content/drive/MyDrive/PosPuc/TCC/datasets/acidentes_ocorrencia/df_preprocessed_new.csv', index=False, encoding='utf-8')